In [7]:
# import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import xgboost as xgb
import shap
import warnings
warnings.filterwarnings('ignore')

# Load cleaned data and prepared splits (reuse from notebook 3)
df = pd.read_csv('amazon_sales_cleaned.csv')
# Recreate features and splits as done in notebook 3 (code omitted for brevity, assume we have X_train, X_test, etc.)

# For demonstration, we focus on two models: Random Forest Regressor for total_revenue and XGBoost Classifier for high_value.



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 1. Encode categorical variables (must match Notebook 3 logic)
le_cat = LabelEncoder()
df['cat_encoded'] = le_cat.fit_transform(df['product_category'])

le_region = LabelEncoder()
df['region_encoded'] = le_region.fit_transform(df['customer_region'])

le_payment = LabelEncoder()
df['payment_encoded'] = le_payment.fit_transform(df['payment_method'])

# 2. Define the exact same feature columns
feature_cols = ['price', 'discount_percent', 'quantity_sold', 'rating', 'review_count', 
                'year', 'month', 'day', 'day_of_week', 'cat_encoded', 'region_encoded', 'payment_encoded']

# 3. Define Features and Target
X = df[feature_cols]
y_rev = df['total_revenue']

# 4. Perform the Train-Test Split (use the same random_state=42)
X_train, X_test, y_train_rev, y_test_rev = train_test_split(X, y_rev, test_size=0.2, random_state=42)


In [8]:
# 1. Hyperparameter Tuning for Random Forest Regressor 

# Use RandomizedSearchCV for efficiency
param_dist_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)
rand_search_rf = RandomizedSearchCV(rf_base, param_dist_rf, n_iter=10, cv=3, scoring='r2', random_state=42, n_jobs=-1)
rand_search_rf.fit(X_train, y_train_rev)   # assuming X_train is original (unscaled) from notebook 3

print("Best RF params:", rand_search_rf.best_params_)
best_rf = rand_search_rf.best_estimator_

# Evaluate
pred_rf = best_rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test_rev, pred_rf))
r2_rf = r2_score(y_test_rev, pred_rf)
print(f"Tuned RF - RMSE: {rmse_rf:.2f}, R2: {r2_rf:.3f}")

Best RF params: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None}
Tuned RF - RMSE: 0.99, R2: 1.000
